# Imports

In [1]:
import praw
import psaw
import numpy
%matplotlib inline
from pylab import *
from pprint import pprint
import pandas as pd
import simplejson as json
from pmaw import PushshiftAPI
import pyspark
import datetime
import os
from itertools import chain
import csv
%config InlineBackend.figure_format='retina'

## Trying KL Divergence

In [22]:
# box =[P(green),P(blue),P(red),P(yellow)]
box_1 = [0.25, 0.33, 0.23, 0.19]
box_2 = [0.21, 0.21, 0.32, 0.26]

import numpy as np
from scipy.special import rel_entr

def kl_divergence(a, b):
	return sum(a[i] * np.log(a[i]/b[i]) for i in range(len(a)))
	
print('KL-divergence(box_1 || box_2): %.3f ' % kl_divergence(box_1,box_2))
print('KL-divergence(box_2 || box_1): %.3f ' % kl_divergence(box_2,box_1))

# D( p || p) =0
print('KL-divergence(box_1 || box_1): %.3f ' % kl_divergence(box_1,box_1))

print("Using Scipy rel_entr function")
box_1 = np.array(box_1)
box_2 = np.array(box_2)

print('KL-divergence(box_1 || box_2): %.3f ' % sum(rel_entr(box_1,box_2)))
print('KL-divergence(box_2 || box_1): %.3f ' % sum(rel_entr(box_2,box_1)))
print('KL-divergence(box_1 || box_1): %.3f ' % sum(rel_entr(box_1,box_1)))



KL-divergence(box_1 || box_2): 0.057 
KL-divergence(box_2 || box_1): 0.056 
KL-divergence(box_1 || box_1): 0.000 
Using Scipy rel_entr function
KL-divergence(box_1 || box_2): 0.057 
KL-divergence(box_2 || box_1): 0.056 
KL-divergence(box_1 || box_1): 0.000 


/var/folders/fh/yf9jwszj6d5_1_xhfz_l6k000000gn/T/ipykernel_24124/3709783919.py:9: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return sum(a[i] * np.log(a[i]/b[i]) for i in range(len(a)))


In [ ]:

confidences = sorted(list(set(loc_pics['confidence'])))[20:-20]

from sklearn.metrics import mutual_info_score

In [ ]:
KL = dict()
for c in confidences:
    array_1 = loc_pics.loc[loc_pics['confidence'] >= c, 'iou'].sort_values().to_list()
    array_2 = loc_pics.loc[loc_pics['confidence'] < c, 'iou'].sort_values().to_list()
        
    if len(array_1) < len(array_2):
        array_2 = np.random.choice(array_2, len(array_1))
    
    else:
        array_1 = np.random.choice(array_1, len(array_2))
    
    KL[c] = mutual_info_score(array_1,array_2)

In [ ]:
max(KL, key=KL.get)

In [5]:
files = os.listdir('/Users/jakobschlierf/Downloads/Voluntarist_meme')

In [9]:
files = [os.path.join('/Users/jakobschlierf/Downloads/Voluntarist_meme', file) for file in files]

In [10]:
files

['/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1607639850.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1643205900.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1583017200.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1613111550.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1610375700.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1593960600.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1604904000.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1640470050.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1626790800.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1637734200.pickle',
 '/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes-1624054950.pickle',
 '/Users/jakobschlierf/Downloads/Voluntaris

In [8]:
def merge_splits(files, subreddit, ptype): # merges the files down
    merge_candidates = []
    for file in files:
        try:
            if os.path.getsize(file) > 459:
                if file[:-18] == subreddit:
                    merge_candidates.append(file)
        except FileNotFoundError:
            pass

    merge_candidates.sort()
    if len(merge_candidates) == 24:
        df = pd.concat([pd.read_pickle(candidate) for candidate in merge_candidates])
        df.to_pickle('/Users/akobschlierf/Desktop/Master/Thesis/Files/Comments/VoluntaristMemes.pickle')
        print(f'{subreddit} merged')
        for i in merge_candidates:
            os.remove(i)
    else:
        print(f'{subreddit} has only {len(merge_candidates)} files to merge. Merging not possible.')

In [13]:
merge_candidates = []
for file in files:
    try:
        if os.path.getsize(file) > 459:
            if file[:-18] == 'VoluntaristMemes':
                merge_candidates.append(file)
    except FileNotFoundError:
        pass

In [17]:
for file in files:
    print(os.path.getsize(file))
    print(file[:-18])

799062
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
164789
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
1617779
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
215004
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
400714
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
2304693
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
1006853
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
107911
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
344347
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
179874
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
316471
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
148441
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
668999
/Users/jakobschlierf/Downloads/Voluntarist_meme/VoluntaristMemes
235911
/Users/jakobschlierf/Downloads/Voluntarist_meme/Volunt

In [18]:
df = pd.concat([pd.read_pickle(candidate) for candidate in files])

In [19]:
len(df)

63699

In [21]:
df.to_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/Comments/VoluntaristMemes.pickle')

In [106]:
with open('../Preprocessing/subreddits.csv', newline='') as f:
    reader = csv.reader(f)
    subreddits = list(reader)

subreddits = list(chain.from_iterable(subreddits))

In [107]:
def identify_missing_files(dir, files):
    missing_files = []
    for file in files:
        if not os.path.exists(os.path.join(dir, file)) or os.path.getsize(os.path.join(dir, file)) <= 459:
            missing_files.append(file)
    return missing_files


def splittimeframe(subreddit, start, end, split): # splits the time into a series of smaller files to pull
    split_list = []
    step = (end - start) / split
    for i in range(split):
            s = int(start + i * step)
            e = int((start + (i + 1) * step) - 1)
            if i == split - 1:
                    e += 86400
            split_list.append(f'{subreddit}-{s}.pickle')

    return split_list

def find_existing_pulls(type, subreddits): #remove existing pulls from subreddits list
    done = os.listdir(f'../../Files/{type}/')
    for i in done:
        done[done.index(i)] = i[:-7]
    res = [i for i in subreddits if i not in done]
    return res


subreddits_not_completed = find_existing_pulls('Submissions', subreddits)

start = int(datetime.datetime(2020, 3, 1).timestamp())
end = int(datetime.datetime(2022, 3, 31).timestamp())
split = 24
missing_parts = []
for subreddit in subreddits_not_completed:
    
    subreddit_parts = splittimeframe(subreddit, start, end, split)
    missing = identify_missing_files(f'../../Files/Submissions/temp/', subreddit_parts)
    for m in missing:
        missing_parts.append(m)

In [109]:
len(missing_parts)

4559

In [79]:
subreddits_not_completed = find_existing_pulls('Submissions', subreddits)

start = int(datetime.datetime(2020, 3, 1).timestamp())
end = int(datetime.datetime(2022, 3, 31).timestamp())
split = 24
missing_parts = []
for subreddit in subreddits_not_completed:
    
    subreddit_parts = splittimeframe(subreddit, start, end, split)
    missing = identify_missing_files(f'../../Files/Submissions/temp/', subreddit_parts)
    for m in missing:
        missing_parts.append(m)


In [84]:
missing_parts.index('PoliticalCompassMemes-1618583250.pickle')

2385

In [90]:
file = open('../../Files/Submissions/temp/missing.csv', "w")
writer = csv.writer(file, delimiter = "\n")
for list_ in missing_parts:
     writer.writerow([list_])
file.close()

In [95]:
print(missing_parts[0][:-18], missing_parts[0][-17:-7])


dankmemes 1583017200


In [17]:
start = int(datetime.datetime(2020, 3, 1).timestamp())
end = int(datetime.datetime(2022, 3, 31).timestamp())
split = 24

In [27]:
def splittimeframe(subreddit, start, end, split):
    split_list = []
    step = (end - start) / split
    for i in range(split):
            s = int(start + i * step)
            e = int((start + (i + 1) * step) - 1)
            if i == split - 1:
                    e += 86400
            split_list.append([subreddit, s, e])

    return split_list

In [126]:
i = 1
for sub, start, end in l:
    print(i, sub, datetime.datetime.fromtimestamp(start).strftime("%m/%d/%Y, %H:%M:%S"), 'end' ,  datetime.datetime.fromtimestamp(end).strftime("%m/%d/%Y, %H:%M:%S"))
    i += 1

1 ivermectin 03/01/2020, 00:00:00 end 04/01/2020, 16:57:29
2 ivermectin 04/01/2020, 16:57:30 end 05/03/2020, 08:54:59
3 ivermectin 05/03/2020, 08:55:00 end 06/04/2020, 00:52:29
4 ivermectin 06/04/2020, 00:52:30 end 07/05/2020, 16:49:59
5 ivermectin 07/05/2020, 16:50:00 end 08/06/2020, 08:47:29
6 ivermectin 08/06/2020, 08:47:30 end 09/07/2020, 00:44:59
7 ivermectin 09/07/2020, 00:45:00 end 10/08/2020, 16:42:29
8 ivermectin 10/08/2020, 16:42:30 end 11/09/2020, 07:39:59
9 ivermectin 11/09/2020, 07:40:00 end 12/10/2020, 23:37:29
10 ivermectin 12/10/2020, 23:37:30 end 01/11/2021, 15:34:59
11 ivermectin 01/11/2021, 15:35:00 end 02/12/2021, 07:32:29
12 ivermectin 02/12/2021, 07:32:30 end 03/15/2021, 23:29:59
13 ivermectin 03/15/2021, 23:30:00 end 04/16/2021, 16:27:29
14 ivermectin 04/16/2021, 16:27:30 end 05/18/2021, 08:24:59
15 ivermectin 05/18/2021, 08:25:00 end 06/19/2021, 00:22:29
16 ivermectin 06/19/2021, 00:22:30 end 07/20/2021, 16:19:59
17 ivermectin 07/20/2021, 16:20:00 end 08/21/2021

In [97]:
l

[['ivermectin', 1583017200, 1585753049],
 ['ivermectin', 1585753050, 1588488899],
 ['ivermectin', 1588488900, 1591224749],
 ['ivermectin', 1591224750, 1593960599],
 ['ivermectin', 1593960600, 1596696449],
 ['ivermectin', 1596696450, 1599432299],
 ['ivermectin', 1599432300, 1602168149],
 ['ivermectin', 1602168150, 1604903999],
 ['ivermectin', 1604904000, 1607639849],
 ['ivermectin', 1607639850, 1610375699],
 ['ivermectin', 1610375700, 1613111549],
 ['ivermectin', 1613111550, 1615847399],
 ['ivermectin', 1615847400, 1618583249],
 ['ivermectin', 1618583250, 1621319099],
 ['ivermectin', 1621319100, 1624054949],
 ['ivermectin', 1624054950, 1626790799],
 ['ivermectin', 1626790800, 1629526649],
 ['ivermectin', 1629526650, 1632262499],
 ['ivermectin', 1632262500, 1634998349],
 ['ivermectin', 1634998350, 1637734199],
 ['ivermectin', 1637734200, 1640470049],
 ['ivermectin', 1640470050, 1643205899],
 ['ivermectin', 1643205900, 1645941749],
 ['ivermectin', 1645941750, 1648763999]]

In [ ]:
files = os.listdir(f'../../Files/{ptype}/temp/')
files.pop(files.index('missing.csv'))

t = [file[:-18] for file in files]

In [115]:
files

['aww-1585753050.pickle',
 'aww-1618583250.pickle',
 'aww-1596696450.pickle',
 'aww-1599432300.pickle',
 'aww-1634998350.pickle',
 'aww-1624054950.pickle',
 'aww-1645941750.pickle',
 'aww-1629526650.pickle',
 'aww-1632262500.pickle',
 'aww-1621319100.pickle',
 'aww-1591224750.pickle',
 'aww-1615847400.pickle',
 'aww-1602168150.pickle',
 'aww-1588488900.pickle',
 'aww-1604904000.pickle',
 'aww-1593960600.pickle',
 'aww-1610375700.pickle',
 'aww-1613111550.pickle',
 'aww-1583017200.pickle',
 'aww-1643205900.pickle',
 'aww-1607639850.pickle',
 'aww-1637734200.pickle',
 'aww-1626790800.pickle',
 'aww-1640470050.pickle']

In [111]:
files = os.listdir('../../TESTMERGE/')


In [113]:
t = [file[:-18] for file in files]

subreddits = []
for i in t:
    if i not in subreddits:
        subreddits.append(i)

In [118]:
def merge_splits(files, subreddit, ptype):
    merge_candidates = []
    for file in files:
        if os.path.getsize(f'../../Files/{ptype}/temp/{file}') > 459:
            if file[:-18] == subreddit:
                # merge_candidates.append(f'../../Files/{ptype}/temp/{file}')
                merge_candidates.append(f'../../TESTMERGE/{file}')

    merge_candidates.sort()
    if len(merge_candidates) == 24:
        df = pd.concat([pd.read_pickle(candidate) for candidate in merge_candidates])
        df.to_pickle(f'../../Files/{ptype}/{subreddit}.pickle')
       
        # for candidate in merge_candidates:
        #     os.remove(candidate)

In [120]:
subreddit = 'aww'
ptype = 'Submissions'
merge_candidates = []
for file in files:
    if os.path.getsize(f'../../Files/{ptype}/temp/{file}') > 459:
        if file[:-18] == subreddit:
            # merge_candidates.append(f'../../Files/{ptype}/temp/{file}')
            merge_candidates.append(f'../../TESTMERGE/{file}')

In [123]:
df = pd.concat([pd.read_pickle(candidate) for candidate in merge_candidates])

In [124]:
len(df)

1251041

## Distribute Pulls test

## Test Validating Pulls

In [9]:
import csv
from itertools import chain
with open('../Preprocessing/subreddits.csv') as f:
    reader = csv.reader(f)
    subreddits = list(reader)
subreddits = list(chain.from_iterable(subreddits))

In [10]:
subreddits

['aspiememes',
 'sciencememes',
 'Anarcho_Capitalism',
 'ConservativeMemes',
 'CoronavirusCirclejerk',
 'JustUnsubbed',
 'theyknew',
 'introvert',
 'southafrica',
 'SurprisingNoOne',
 'AnythingGoesNews',
 'GoldandBlack',
 'PositiveNewsCovid19',
 'RuralNewsNetwork',
 'Anticonsumption',
 'GreenAndPleasant',
 'January6',
 'LandlordLove',
 'LateStageImperialism',
 'OliverMarkusMalloy',
 'Palestine',
 'lebanon',
 'tech',
 'wholesome',
 '196',
 'ToiletPaperUSA',
 'TrollFaceIncident',
 '2meirl42meirl4meirl',
 'wholesomegreentext',
 'ANormalDayInAmerica',
 'AOC',
 'TheFightThatMatters',
 'BeansInThings',
 'WalmartCelebrities',
 'WtWBBot',
 'thanksihateit',
 'BadChoicesGoodStories',
 'InsaneProtestors',
 'SocialJusticeInAction',
 'TheNarcoblog701',
 'alberta',
 'islam',
 'worldnewsvideo',
 'AmericanFascism2020',
 'AreTheStraightsOK',
 'Astuff',
 'AskWomenOfColorOver30',
 'anime_titties',
 'AutisticPride',
 'Moronavirus',
 'BoycottChina',
 'BreadTube',
 'thedavidpakmanshow',
 'COMPLETEANARCHY',


In [13]:
type = 'Submissions'
done = os.listdir(f'../../Files/{type}/')
for i in done:
    done[done.index(i)] = i[:-7]
res = [i for i in subreddits if i not in done]

In [16]:
print(len(subreddits), len(done))

705 23


In [15]:
res = [i for i in subreddits if i not in done]

In [17]:
print(len(res))

685


## Tests for Identifying subs

In [40]:
def aggregate(df): #aggregate over the subreddits so that we get a list with subreddit - crosspost from and counts
    
    df2 = df.groupby(['subreddit','crosspost_parent']).agg({'subreddit_subscribers': 'mean', 'crosspost_parent_subs': 'mean' , 'author' : 'count', 'crosspost_parent_num': 'sum'}).reset_index()
    df2.rename(columns = {'author':'count',}, inplace = True)
    df2['crosspost_parent_subs'] = df2['crosspost_parent_subs'].fillna(0)
    df2['crosspost_parent_subs'] = df2['crosspost_parent_subs'].astype(int)

    imp = df2.groupby(['crosspost_parent']).agg({'subreddit': 'count', 'count': 'sum'})
    imp = imp.rename(columns ={'count': 'total'}) 
    imp.drop(['subreddit'], axis=1, inplace=True)
    df3 = df2.merge(imp, on='crosspost_parent', how='left')

    print('aggregated')
    return df3

In [41]:
def get_crosspost_child(df, outfile, depth_lim): #find the crosspost children of posts from the observed subreddits
    # api = PushshiftAPI()
    # df = df[df['num_crossposts'] > 0] # We do this by looking for posts that have been crossposted
    # t = df.groupby(['url']).agg({'num_crossposts': 'sum', 'id':  'max'}).reset_index()
    # urls = list(t['url']) # then collect the url of posts / attached links / pictures

    # df = pd.DataFrame(columns=['id', 'url', 'title', 'subreddit', 'selftext', 'subreddit_subscribers',
    #     'num_crossposts', 'crosspost_parent', 'created_utc', 'author', 'num_comments', 'score'])

    # for j in urls:
    #     results2 = api.search_submissions(
    #         url = j, # and search for them using the pushshift api
    #         filter=[ 'id', 'url', 'title', 'subreddit', 'selftext', 'subreddit_subscribers',
    #         'num_crossposts', 'crosspost_parent', 'created_utc', 'author', 'num_comments', 'score']) 
    #     temp = pd.DataFrame([thing for thing in results2])
    #     df = pd.concat([df, temp])

    # df.to_pickle(f'../../Files/{outfile}_raw_child_{depth_lim}.pickle')
    df2 = df[df['num_crossposts'] > 0].reset_index(drop=True) #split into parent posts (number of crossposts > 0 )
    df3 = df[df['num_crossposts'] == 0].reset_index(drop=True) # and child posts (number of crossposts == 0)
    df3 = df3[df3['crosspost_parent'].notna()] # Drop all children where field for parent is empty
    df3['crosspost_parent']  = df3['crosspost_parent'].apply(lambda x: x[3:])
    df2.drop('crosspost_parent', axis=1, inplace=True)
    df2 = df2.rename(columns ={'subreddit':'crosspost_parent', 'subreddit_subscribers': 'crosspost_parent_subs', 'num_crossposts': 'crosspost_parent_num'})
    df4 = df2.merge(df3, left_on='id', right_on='crosspost_parent', suffixes= ('','_y'))

    df4 = df4[(df4['crosspost_parent'].str.startswith('u_') == False) & (df4['subreddit'].str.startswith('u_') == False)] # get rid off users as subreddit
    df5 = aggregate(df4)
    
    df
    print(f'identified {len(df4)} children')
    df5.to_pickle(f'../../Files/{outfile}_cross_child_{depth_lim}.pickle') # save file to avoid straining the API
    return df5

In [4]:
df = pd.read_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/test_0613_raw_child_2.pickle')


In [42]:
df.head()

,id,url,title,subreddit,selftext,subreddit_subscribers,num_crossposts,crosspost_parent,created_utc,author,num_comments,score
0,n6yugl,https://pdf.math.dev/,Beautiful PDFs from HTML,u_dweeed,,0,0,t3_n6yjsj,1620395121,dweeed,0,1
1,n6yjsj,https://pdf.math.dev/,Beautiful PDFs from HTML,webdev,,821591,0,NaN,1620394253,speckz,19,1
2,n6yjm4,https://pdf.math.dev/,Beautiful PDFs from HTML,web_design,,536815,0,NaN,1620394240,speckz,6,1
3,n6yjju,https://pdf.math.dev/,Beautiful PDFs from HTML,Frontend,,135880,0,NaN,1620394235,speckz,0,1
4,mli1c7,https://pdf.math.dev,Guide (and example code) to producing beautifu...,css,,82004,0,NaN,1617733009,RentGreat8009,2,1


In [43]:
df2 = get_crosspost_child(df, 'serverfix', 2 )

aggregated
identified 15950 children


In [44]:
df2.head()

,subreddit,crosspost_parent,subreddit_subscribers,crosspost_parent_subs,count,crosspost_parent_num,total
0,0sanitymemes,interestingasfuck,9076.0,7595870,1,16,541
1,111bookmarks,ATBGE,1.0,1418326,1,2,481
2,1200isfineIGUESSugh,peopleofwalmart,50073.0,409263,1,1,13
3,13or30,HumansBeingBros,406092.0,2215179,1,7,144
4,13or30,sadcringe,404388.0,924672,1,4,63


In [45]:
df3 = pd.read_pickle('../../Files/test_0613_cross_parent_2.pickle')

In [46]:
df3.head()

,subreddit,crosspost_parent,subreddit_subscribers,crosspost_parent_subs,count,crosspost_parent_num,total
0,6thForm,2meirl4meirl,35791.0,1268402,1,3,111
1,6thForm,6thForm,37984.0,37984,4,8,4
2,6thForm,Advice,37341.0,512881,1,1,11
3,6thForm,AlevelGeog,41873.0,159,1,1,1
4,6thForm,ApplyingToCollege,38328.5,325892,2,2,6


In [48]:
df4 = pd.concat([df3, df2])

In [51]:
df2.shape

(8863, 7)

In [49]:
df4.shape

(30629, 7)

In [52]:
df4.to_pickle('../../Files/test_0613_cross.pickle')

In [78]:
CLIENT_ID = '5FpjYNUrv93rDqflbNE-6w'
CLIENT_SECRET = '5ZITWnmNd6_qCbPXDB90xmOaIMo9_w'
USER_NAME = 'theonejay96'
SCRIPT_NAME = 'Data\ Science\ Thesis'
LOGIN_PASSWORD = 'vejkyz-hyBfez-4cinte'

reddit =  praw.Reddit(
    client_id= CLIENT_ID ,
    client_secret= CLIENT_SECRET ,
    user_agent= SCRIPT_NAME,
    username= USER_NAME,
    password= LOGIN_PASSWORD)

api = PushshiftAPI()

In [37]:
api = PushshiftAPI()

In [82]:
gen2 = api.search_submissions(
    filter=[ 'id', 'url', 'title', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'crosspost_parent', 'crosspost_parent_list', 'created_utc', 'author'],
    limit= 100)

temp = pd.DataFrame([thing for thing in gen2])
temp.head()

Not all PushShift shards are active. Query results may be incomplete.


,author,created_utc,id,num_crossposts,subreddit,subreddit_id,subreddit_subscribers,title,url,crosspost_parent,crosspost_parent_list
0,David_McTravord555,1654625979,v73j9f,0,worldbuilding,t5_2rd6n,905089,In my universe where magical creatures (like v...,https://www.reddit.com/r/worldbuilding/comment...,NaN,NaN
1,Antdryem_,1654625979,v73j9d,0,Dublin,t5_2r1hz,76494,Getting to live in Dublin,https://www.reddit.com/r/Dublin/comments/v73j9...,NaN,NaN
2,Spnkmeimbd,1654625979,v73j9a,0,sideboobandass,t5_2pam8b,3271,Come keep me warm,https://i.redd.it/y7heujw9uy391.jpg,t3_v5yhln,"[{'all_awardings': [], 'allow_live_comments': ..."
3,johnevepierrot,1654625979,v73j99,0,Bromance2,t5_3h6m2l,383,Looking for Connections,https://www.reddit.com/r/Bromance2/comments/v7...,NaN,NaN
4,pirl_opden,1654625979,v73j97,0,HiTMAN,t5_2rn0x,134437,"Another JIRA ticket,another set of funerals",https://v.redd.it/w7e46i2bo8491,NaN,NaN


In [80]:
g = [ting for ting in gen2]

In [79]:

temp = pd.DataFrame([thing for thing in gen2])
temp.head()

""


In [81]:
g

[]

In [207]:
df = pd.read_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/test0606__cross.pickle')

In [208]:
df.head()

,subreddit,subreddit_id,crosspost_parent,crosspost_parent_id,subreddit_subscribers,crosspost_parent_subs,count,crosspost_parent_num,total
0,CovidVaccinated,t5_3gppqs,MapPorn,t5_2si92,4351.0,1252559,1,1,1
1,CovidVaccinated,t5_3gppqs,dataisbeautiful,t5_2tk95,4196.0,15347693,1,2,1
2,DebateVaccines,t5_3jwf7,AntiVaxxers,t5_384mz,2354.5,27397,2,26,2
3,DebateVaccines,t5_3jwf7,ConspiracyUltra,t5_3flq3y,2711.869565,236,23,23,23
4,DebateVaccines,t5_3jwf7,Coronavirus,t5_2x4yx,2585.5,2365057,4,5,12


In [4]:
df = pd.read_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/test0606__ratio_temp2')
df.head()

,subreddit,crosspost_parent,crosspost_child,total,crosspost_parent_%,crosspost_child_%
0,ABoringDystopia,857,10,2393,0.358128,0.004179
1,ATBGE,14,408,2400,0.005833,0.170000
2,ActualPublicFreakouts,823,49,2382,0.345508,0.020571
3,AgainstHateSubreddits,1,7,1908,0.000524,0.003669
4,AlternativeHealth,42,0,245,0.171429,0.000000


In [19]:
df = pd.read_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/test0606__cross_child.pickle')
df.head()

,subreddit,subreddit_id,crosspost_parent,crosspost_parent_id,subreddit_subscribers,crosspost_parent_subs,count,crosspost_parent_num,total
0,13or30,t5_3mzz4,Conservative,t5_2qh6p,406093.0,717882,1,2,89
1,1950s,t5_2suyd,interestingasfuck,t5_2qhsa,6798.0,7474756,1,3,781
2,197,t5_3enhu,DankLeft,t5_11ga7z,16225.0,157790,5,5,63
3,2007scape,t5_2wbww,ThatsInsane,t5_swxxz,557307.0,862697,1,22,164
4,2007scape,t5_2wbww,funny,t5_2qh33,550208.0,35286028,1,11,146


In [25]:
df.shape

(4881, 9)

In [22]:
# df4 = df4[(df4['crosspost_parent'].str.contains('u_') == False)]
# df2 = df[(df['crosspost_parent'].str.startswith('u_') == False)]

df2 = df[(df['crosspost_parent'].str.startswith('u_') == False) & (df['subreddit'].str.startswith('u_') == False)]

In [24]:
df2.shape

(4102, 9)

In [197]:
df2 = df[df['crosspost_parent_list'].notna()].reset_index()
df2 = df2[df2['crosspost_parent_list'].str.len() != 0] #sometimes, this field contains an empty list

df2['t'] = df2['crosspost_parent_list'].apply(lambda x: dict(x[0])) # Pull crosspost_from information from field 'crosspost_parent_list' (which is in a json format)
df2['crosspost_parent'] = df2['t'].apply(lambda x: x['subreddit'])  # Using the field has the advantage that we can deal with deleted posts which we could not find using the reddit API.
df2['crosspost_parent_id'] = df2['t'].apply(lambda x: x['subreddit_id'])
df2['crosspost_parent_subs'] = df2['t'].apply(lambda x: x['subreddit_subscribers'] )
df2['crosspost_parent_num'] = df2['t'].apply(lambda x: x['num_crossposts'] )


In [198]:
df2.head()

,index,id,url,title,subreddit,subreddit_id,subreddit_subscribers,crosspost_parent,crosspost_parent_list,num_crossposts,created_utc,author,created,t,crosspost_parent_id,crosspost_parent_subs,crosspost_parent_num
0,3,l9ncqc,https://i.redd.it/i214kiu1jqe61.jpg,What’s better than freezing to death,ABoringDystopia,t5_3c6l1,514355,WhitePeopleTwitter,"[{'all_awardings': [], 'allow_live_comments': ...",0,1612130867,doctornowzaradan,1.612124e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_35n7t,2011831,1
1,5,l9m64s,https://v.redd.it/28r4g0xudne61,How are those $2000 relief checks coming along?,ABoringDystopia,t5_3c6l1,514354,NewDealAmerica,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612127619,ProctorZeuss,1.612120e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_2xl4f0,43387,6
2,6,l9kxsi,https://i.redd.it/d0u3bosh3me61.jpg,Jimmy is a fucking badass,ABoringDystopia,t5_3c6l1,514347,nextfuckinglevel,[{'all_awardings': [{'award_sub_type': 'PREMIU...,0,1612124273,[deleted],1.612117e+09,{'all_awardings': [{'award_sub_type': 'PREMIUM...,t5_m0bnr,3622511,25
3,7,l9kltk,https://i.redd.it/kbybo0r1ppe61.jpg,School staff is one of the most important peop...,ABoringDystopia,t5_3c6l1,514347,MadeMeSmile,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612123409,AnoonymouseChocobo,1.612116e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_2uqcm,2852714,3
4,9,l9kepr,https://i.redd.it/k7scw0fiqpe61.jpg,Wtf children are not fined for late loans in m...,ABoringDystopia,t5_3c6l1,514346,wholesomememes,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612122905,Damroth,1.612116e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_3gcwj,8871710,1


In [206]:
df3 = df2[(df2['crosspost_parent'].str.contains('u_') == False)]
df3

,index,id,url,title,subreddit,subreddit_id,subreddit_subscribers,crosspost_parent,crosspost_parent_list,num_crossposts,created_utc,author,created,t,crosspost_parent_id,crosspost_parent_subs,crosspost_parent_num
0,3,l9ncqc,https://i.redd.it/i214kiu1jqe61.jpg,What’s better than freezing to death,ABoringDystopia,t5_3c6l1,514355,WhitePeopleTwitter,"[{'all_awardings': [], 'allow_live_comments': ...",0,1612130867,doctornowzaradan,1.612124e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_35n7t,2011831,1
1,5,l9m64s,https://v.redd.it/28r4g0xudne61,How are those $2000 relief checks coming along?,ABoringDystopia,t5_3c6l1,514354,NewDealAmerica,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612127619,ProctorZeuss,1.612120e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_2xl4f0,43387,6
2,6,l9kxsi,https://i.redd.it/d0u3bosh3me61.jpg,Jimmy is a fucking badass,ABoringDystopia,t5_3c6l1,514347,nextfuckinglevel,[{'all_awardings': [{'award_sub_type': 'PREMIU...,0,1612124273,[deleted],1.612117e+09,{'all_awardings': [{'award_sub_type': 'PREMIUM...,t5_m0bnr,3622511,25
3,7,l9kltk,https://i.redd.it/kbybo0r1ppe61.jpg,School staff is one of the most important peop...,ABoringDystopia,t5_3c6l1,514347,MadeMeSmile,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612123409,AnoonymouseChocobo,1.612116e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_2uqcm,2852714,3
4,9,l9kepr,https://i.redd.it/k7scw0fiqpe61.jpg,Wtf children are not fined for late loans in m...,ABoringDystopia,t5_3c6l1,514346,wholesomememes,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612122905,Damroth,1.612116e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_3gcwj,8871710,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8787,95,ltoqoj,https://i.redd.it/tb78uxank0k61.jpg,Please join us #Condemns_the_terrorist_activit...,UnitedNations,t5_2s0h9,6426,myanmar,"[{'all_awardings': [], 'allow_live_comments': ...",0,1614434546,memelgi,1.614427e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_2rt95,8938,1
8788,96,lto7a1,https://v.redd.it/scqr8x6vyyj61,It's no longer safe for us. She is just a frui...,UnitedNations,t5_2s0h9,6425,myanmar,"[{'all_awardings': [], 'allow_live_comments': ...",0,1614432753,memelgi,1.614426e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_2rt95,8937,1
8789,97,lto68o,https://i.redd.it/wi6qozresyj61.jpg,A pregnant woman was arrested by myanmar brutu...,UnitedNations,t5_2s0h9,6425,myanmar,"[{'all_awardings': [], 'allow_live_comments': ...",0,1614432653,memelgi,1.614425e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_2rt95,8937,1
8790,98,m5cvc5,/r/LibertarianPartyUSA/comments/m5cpqj/liberta...,Libertarian startup city Próspera has broken g...,GoldandBlack,t5_3ffci,79304,LibertarianPartyUSA,"[{'all_awardings': [], 'allow_live_comments': ...",0,1615783183,nixfu,1.615776e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_3epv4,7285,1


In [200]:
df2 = df2[df2['crosspost_parent'] != 'u_']

In [150]:
t = df.iloc[3]['crosspost_parent_list']

In [155]:
t[0]['subreddit'][0:2]

'Wh'

In [143]:
df['t'] = df['crosspost_parent_list'].apply(lambda x: dict(x[0]))


TypeError: 'float' object is not subscriptable

In [132]:

df23 = df[df['crosspost_parent'].notna()]
# df23 = df23[df23['crosspost_parent'].apply(lambda x: x[0:1]) != 't3']
# df23.head()

In [126]:
df23.shape

(8792, 12)

In [82]:
df4 = df[df['crosspost_parent'].apply(lambda x: x[0:2]) != 'u_']
df4.shape

TypeError: 'float' object is not subscriptable

In [71]:

df3 = df2.groupby('subreddit').agg({'crosspost_parent': 'count', 'num_crossposts': lambda x: np.count_nonzero(x), 'subreddit_subscribers' : 'count'}).reset_index()
df3 = df3.rename(columns ={'subreddit_subscribers':'total', 'num_crossposts': 'crosspost_child'})

In [72]:
df3.sort_values('crosspost_parent',ascending=False)

,subreddit,crosspost_parent,crosspost_child,total
135,u_rklokh,900,0,900
9,Astuff,503,9,755
103,covidskepticscanada,347,15,603
0,ABoringDystopia,325,9,900
57,NEWPOLITIC,311,22,900
...,...,...,...,...
115,montreal,0,3,897
122,privacy,0,13,891
116,news,0,19,900
120,pcgaming,0,4,900


In [73]:
df3['crosspost_parent_%'] = df3['crosspost_parent'] / df3['total']
df3['crosspost_child_%'] = df3['crosspost_child'] / df3['total']

In [78]:
df3['crosspost_parent_%'].mean()

0.20008680636389625

In [79]:
df3['crosspost_child_%'].mean()

0.019490443440403512

In [255]:
df = pd.DataFrame(columns=['id', 'url', 'title','subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'crosspost_parent', 'created_utc', 'author'])
for i in urls:
    gen2 = api.search_submissions(
    url = i ,
    filter=[ 'id', 'url', 'title', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'crosspost_parent', 'created_utc', 'author'],
    limit = 100) #limit to avoid going over the rate limit.
    results2 = list(gen2)
    temp = pd.DataFrame([thing.d_ for thing in results2])
    df = pd.concat([df, temp])

df2 = df[df['num_crossposts'] > 0].reset_index(drop=True)
df3 = df[df['num_crossposts'] == 0].reset_index(drop=True)
df3 = df3[df3['crosspost_parent'].notna()]
df3['crosspost_parent']  = df3['crosspost_parent'].apply(lambda x: x[3:])
df2.drop('crosspost_parent', axis=1, inplace=True)
df2 = df2.rename(columns ={'subreddit':'crosspost_parent','subreddit_id': 'crosspost_parent_id', 'subreddit_subscribers': 'crosspost_parent_subs', 'num_crossposts': 'crosspost_parent_num'})
df4 = df2.merge(df3, left_on='id', right_on='crosspost_parent', suffixes= ('','_y'))
df5 = df4.groupby(['subreddit','subreddit_id','crosspost_parent', 'crosspost_parent_id']).agg({'subreddit_subscribers': 'mean', 'crosspost_parent_subs': 'mean' , 'author' : 'count', 'crosspost_parent_num': 'sum'}).reset_index()
df5.rename(columns = {'author':'count',}, inplace = True)
df5['crosspost_parent_subs'] = df5['crosspost_parent_subs'].astype(int)


/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


KeyboardInterrupt: 

In [3]:
# df4.to_pickle('../../Files/merge_test.pickle')
df4 = pd.read_pickle('../../Files/test_child.pickle')

In [6]:
df4.head()
df = df4

In [7]:
df2 = df[df['num_crossposts'] > 0].reset_index(drop=True)
df3 = df[df['num_crossposts'] == 0].reset_index(drop=True)
df3 = df3[df3['crosspost_parent'].notna()]
df3['crosspost_parent']  = df3['crosspost_parent'].apply(lambda x: x[3:])
df2.drop('crosspost_parent', axis=1, inplace=True)
df2 = df2.rename(columns ={'subreddit':'crosspost_parent','subreddit_id': 'crosspost_parent_id', 'subreddit_subscribers': 'crosspost_parent_subs', 'num_crossposts': 'crosspost_parent_num'})
df4 = df2.merge(df3, left_on='id', right_on='crosspost_parent', suffixes= ('','_y'))
df5 = df4.groupby(['subreddit','subreddit_id','crosspost_parent', 'crosspost_parent_id']).agg({'subreddit_subscribers': 'mean', 'crosspost_parent_subs': 'mean' , 'author' : 'count', 'crosspost_parent_num': 'sum'}).reset_index()
df5.rename(columns = {'author':'count',}, inplace = True)
df5['crosspost_parent_subs'] = df5['crosspost_parent_subs'].astype(int)

In [315]:
df5 = df2.merge(df3, left_on='id', right_on='crosspost_parent', suffixes= ('','_y'))


In [324]:
df5 = df4.groupby(['subreddit','subreddit_id','crosspost_parent', 'crosspost_parent_id']).agg({'subreddit_subscribers': 'mean', 'crosspost_parents_subs': 'mean' , 'author' : 'count', 'crosspost_parent_num': 'sum'}).reset_index()
# df3.rename(columns = {'author':'count',}, inplace = True)
# df3['crosspost_from_subs'] = df3['crosspost_from_subs'].astype(int)
    

In [47]:
k = df5.columns
for j in k:
    print(j)

subreddit
subreddit_id
crosspost_parent
crosspost_parent_id
subreddit_subscribers
crosspost_parent_subs
count
crosspost_parent_num


In [14]:
s = df5['subreddit'].drop_duplicates()
s.append(df5['crosspost_parent'].drop_duplicates())
s

/var/folders/fh/yf9jwszj6d5_1_xhfz_l6k000000gn/T/ipykernel_28294/726648991.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s.append(df5['crosspost_parent'].drop_duplicates())


0         ABoringDystopia
2            AconspiracyA
3       AlternativeHealth
5      Anarcho_Capitalism
7        AnythingGoesNews
              ...        
163          vacci_nation
164           vandwellers
165              walkaway
166     whatsreallygoinon
167             worldnews
Name: subreddit, Length: 126, dtype: object

In [17]:

for i in s:
    print(i)

ABoringDystopia
AconspiracyA
AlternativeHealth
Anarcho_Capitalism
AnythingGoesNews
Astuff
BidenLies
COVID
CeticosRespostaCovid
ChurchOfCOVID
CoVaxSkeptical
ConservativeMemes
ConspiracyUltra
CoronaKritiker
CoronaNC
CoronavirusCirclejerk
CoronavirusColorado
CoronavirusOntario
CoronavirusOregon
CoronavirusToronto
CoronavirusUK
CoronavirusUS
CoronavirusVariants
CovidIsACult
CovidVaccinated
DankLeft
DebateVaccine
DebateVaccines
DescentIntoTyranny
DrFauci
EndTheLockdowns
EnoughAntifaSpam
FUCKCHINAVIRUS
FuckFuckMasks
GreatReject
HillaryForPrison
JustUnsubbed
LeopardsAteMyFace
LibertarianLGBTQ
LockdownCriticalLeft
LockdownSkepticismCAN
LoveForDronePilots
Mujico
NEWPOLITIC
NoNewNormal
NoNewNormalBan
NoNoNewNormal
NoShitSherlock
NolibsWatch
OpenTheSchools
PeoplesPartyofCanada
Republican
RepublicanValues
RuralNewsNetwork
SatanicRitualAbuse
Slovakia
SurprisingNoOne
Thailand
TheBeliefInstinct
TheFightThatMatters
TheGreatResistance
ThePeoplesPower
TopConspiracy
TrueAntiVaccination
TruthLeaks
Unbiase

In [51]:
import numpy as np
np.savetxt("../../Files/depth1.csv", 
           s,
           delimiter =", ", 
           fmt ='% s')

In [35]:
t = df5['subreddit'].drop_duplicates().to_list()

In [39]:
t = df5['subreddit'].drop_duplicates().to_list()
t.extend(df5['crosspost_parent'].drop_duplicates().to_list())
res = []
for i in t:
    if i not in res and i not in subreddits:
        res.append(i)

AttributeError: 'list' object has no attribute 'drop_duplicates'

In [41]:

subreddits = ['DebateVaccines', 'CovidVaccinated', 'Vaccine', 'Coronavirus', 'LockdownSkepticism', 'HermanCainAward', 'NoNewNormal']
res = []
for i in t:
    if i not in res and i not in subreddits:
        res.append(i)

In [44]:
print(len(t), len(res))

177 129


In [45]:
res

['ABoringDystopia',
 'AconspiracyA',
 'AlternativeHealth',
 'Anarcho_Capitalism',
 'AnythingGoesNews',
 'Astuff',
 'BidenLies',
 'COVID',
 'COVID19_support',
 'CeticosRespostaCovid',
 'ChurchOfCOVID',
 'CoVaxSkeptical',
 'ConservativeMemes',
 'ConspiracyUltra',
 'CoronaKritiker',
 'CoronaNC',
 'CoronavirusCirclejerk',
 'CoronavirusColorado',
 'CoronavirusOntario',
 'CoronavirusOregon',
 'CoronavirusToronto',
 'CoronavirusUK',
 'CoronavirusUS',
 'CoronavirusVariants',
 'CovidIsACult',
 'DankLeft',
 'DebateVaccine',
 'DescentIntoTyranny',
 'DrFauci',
 'EndTheLockdowns',
 'EnoughAntifaSpam',
 'FUCKCHINAVIRUS',
 'FuckFuckMasks',
 'GoldandBlack',
 'GreatReject',
 'Health',
 'HillaryForPrison',
 'JustUnsubbed',
 'LeopardsAteMyFace',
 'Liberate_Canada',
 'LibertarianLGBTQ',
 'LockdownCriticalLeft',
 'LockdownSkepticismCAN',
 'LoveForDronePilots',
 'Mujico',
 'NEWPOLITIC',
 'NoNewNormalBan',
 'NoNoNewNormal',
 'NoShitSherlock',
 'NolibsWatch',
 'OpenTheSchools',
 'PeoplesPartyofCanada',
 'Repu

In [2]:
def importance(df): #calculate the importance (similar to tf-idf)
    # OPEN
    # - make work with Crosspost from
    imp = df.groupby(['crosspost_from']).agg({'subreddit': 'count', 'count': 'sum', 'crosspost_from_num' : 'sum'})
    imp = imp.rename(columns ={'count': 'total'}) 
    imp.drop(['subreddit'], axis=1, inplace=True)
    df2 = df.join(imp, on='crosspost_from', how='left')
    return df2

In [75]:
df2 = pd.read_pickle('../../Files/2021-01-02.pickle')

In [68]:
df3 = df2[df2['crosspost_parent_list'].str.len() == 0]

In [76]:
df2.head()

,index,subreddit,subreddit_id,subreddit_subscribers,crosspost_parent,crosspost_parent_list,created_utc,author,created,t,crosspost_from,crosspost_from_id,crosspost_from_subs,crosspost_from_num
0,0,DebateVaccines,t5_3jwf7,2347,t3_l9iiqp,"[{'all_awardings': [], 'allow_live_comments': ...",1612117931,[deleted],1.612111e+09,"{'all_awardings': [], 'allow_live_comments': F...",conspiracy,t5_2qh4r,1455036,1
1,3,DebateVaccines,t5_3jwf7,2325,t3_l81oft,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,1612056522,doingbasiclifeprep,1.612049e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,IAmA,t5_2qzb6,20780922,1
2,15,DebateVaccines,t5_3jwf7,2317,t3_l7apag,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,1611915334,EuCleo,1.611908e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,Coronavirus,t5_2x4yx,2363477,1
3,17,DebateVaccines,t5_3jwf7,2314,t3_l77ln3,"[{'all_awardings': [], 'allow_live_comments': ...",1611865305,dannylenwinn,1.611858e+09,"{'all_awardings': [], 'allow_live_comments': F...",ForUnitedStates,t5_2i3wxn,2757,26
4,18,DebateVaccines,t5_3jwf7,2314,t3_l746cr,"[{'all_awardings': [], 'allow_live_comments': ...",1611857406,dannylenwinn,1.611850e+09,"{'all_awardings': [], 'allow_live_comments': F...",ForUnitedStates,t5_2i3wxn,2754,17


In [ ]:
submission = reddit.submission('t3_p8ibgd')
submission.title

In [40]:
r['crosspost_parent_list'][0]

IndexError: list index out of range

In [26]:
df2 = df2[df2['crosspost_parent'].notna()].reset_index()

In [52]:

df2.to_pickle(f'../../Files/test.pickle')

IndexError: list index out of range

## Encoding of times

In [ ]:
def base36encode(number, alphabet='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
    """Converts an integer to a base36 string."""
    if not isinstance(number, (int)):
        raise TypeError('number must be an integer')

    base36 = ''
    sign = ''

    if number < 0:
        sign = '-'
        number = -number

    if 0 <= number < len(alphabet):
        return sign + alphabet[number]

    while number != 0:
        number, i = divmod(number, len(alphabet))
        base36 = alphabet[i] + base36

    return sign + base36


In [ ]:
import datetime
datetime_obj = datetime.datetime.fromtimestamp(1642024930).strftime('%Y-%m-%d %H:%M:%S')
print(datetime_obj)

## PySpark Test

In [6]:
from pyspark import SparkConf, SparkContext 
sc = SparkContext("local[4]", 'spark_test')





ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=spark_test, master=local[4]) created by __init__ at /tmp/ipykernel_320226/2751826978.py:2 

In [36]:
lines = sc.textFile("Files/RS_2005-07.bz2", )

In [42]:
# lines.map(lambda x: x[0]).take(1)
lines.take(1)

['{"archived":true,"author":"zaz","author_flair_background_color":null,"author_flair_css_class":null,"author_flair_richtext":[],"author_flair_text":null,"author_flair_text_color":null,"author_flair_type":"text","brand_safe":true,"can_gild":true,"contest_mode":false,"created_utc":1120181161,"distinguished":null,"domain":"cnn.com","edited":false,"gilded":0,"hidden":false,"hide_score":false,"id":"204","is_crosspostable":true,"is_reddit_media_domain":false,"is_self":false,"is_video":false,"link_flair_css_class":null,"link_flair_richtext":[],"link_flair_text":null,"link_flair_text_color":"dark","link_flair_type":"text","locked":false,"media":null,"media_embed":{},"no_follow":false,"num_comments":0,"num_crossposts":0,"over_18":false,"parent_whitelist_status":"all_ads","permalink":"\\/r\\/reddit.com\\/comments\\/204\\/cnncom_dj_admits_false_tale_about_missing_teen\\/","rte_mode":"markdown","score":4,"secure_media":null,"secure_media_embed":{},"selftext":"","send_replies":true,"spoiler":false,

In [2]:
from multiprocessing import Pool

def f(x):
    return x*x


with Pool(5) as p:
    print(p.map(f, [1, 2, 3]))


Process SpawnPoolWorker-11:
Process SpawnPoolWorker-10:
Process SpawnPoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
Traceback (most recent call last):
AttributeError: Can't get attribute 'f' on <module '__main__' (built-in)>
  File "/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/multiprocessing/process.py", line 315, in

KeyboardInterrupt: 

## Profiling

In [1]:
import cProfile

In [2]:
import pstats
p = pstats.Stats('../../Files/logs/Preproc')


In [3]:
p.sort_stats('name').print_stats(10)

Tue Aug 23 17:57:46 2022    ../../Files/logs/Preproc

         10130611 function calls (9901322 primitive calls) in 24.725 seconds

   Ordered by: function name
   List reduced from 17795 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {blis.cy.init}
     3674    0.000    0.000    0.000    0.000 {built-in method _CheckCalledFromGeneratedFile}
    19383    0.043    0.000    0.044    0.000 {built-in method __new__ of type object at 0x10b5b1a50}
     1252    0.007    0.000    0.007    0.000 {built-in method _abc._abc_init}
     3952    0.003    0.000    0.006    0.000 {built-in method _abc._abc_instancecheck}
       80    0.000    0.000    0.001    0.000 {built-in method _abc._abc_register}
 3181/880    0.006    0.000    0.007    0.000 {built-in method _abc._abc_subclasscheck}
        1    0.000    0.000    0.000    0.000 {built-in method _abc.get_cache_token}
        9    0.000    0.000  

In [4]:
p.sort_stats('tottime').print_stats(10)

Tue Aug 23 17:57:46 2022    ../../Files/logs/Preproc

         10130611 function calls (9901322 primitive calls) in 24.725 seconds

   Ordered by: internal time
   List reduced from 17795 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  359/355    5.343    0.015    5.352    0.015 {built-in method _imp.create_dynamic}
     4448    3.471    0.001    3.471    0.001 {built-in method io.open_code}
     8685    1.104    0.000    1.123    0.000 thinc/backends/numpy_ops.pyx:90(gemm)
    18772    0.810    0.000    0.810    0.000 {method 'read' of '_io.BufferedReader' objects}
     4448    0.497    0.000    0.497    0.000 {built-in method marshal.loads}
     2320    0.460    0.000    4.773    0.002 spacy/pipeline/trainable_pipe.pyx:40(__call__)
        1    0.449    0.449    0.449    0.449 {built-in method mkl._py_mkl_service.get_version}
     6890    0.411    0.000    0.531    0.000 {built-in method io.open}
    37346    0.397    0.000    

In [6]:
import pandas as pd
df = pd.read_pickle('../../Files/Submissions/profile_target/AwakenTheSheep.pickle')

In [8]:
df.cleanTitle

0     television input affect immediate mindframe hu...
1     importance daily affirmation success   jack ca...
2     censorware internet news content personalize t...
3                      american city shell company rich
4                january [ NUM]th capitol protest stage
5     fight right let deep state manipulate voice st...
6                            patient cure customer lose
7      funny thing people scoff mere idea actually true
8            psa unity amp love powerful force universe
9                         elite sacrifice animal moloch
10                        elite sacrifice animal moloch
11                            u_p[num]tr[num]ot suspend
12    illuminati want exterminate [ NUM]rd populatio...
13                               statism mental illness
14                                            mean like
15            join rantianimalabuseritual raconspiracya
16    recommendationadd post flair animal sacrificea...
17    recommendationif rawakenthesheep member mi

## Test Identifying

In [8]:
def get_files(dir):
    files = []
    for file in os.listdir(dir):
        if file.endswith(".pickle") and os.path.getsize(os.path.join(dir, file)) > 459:
            files.append(os.path.join(dir, file))
    return files.sort()


def identify_missing_files(dir, files):
    missing_files = []
    for file in files:
        if not os.path.exists(os.path.join(dir, file)) or os.path.getsize(os.path.join(dir, file)) <= 459:
            missing_files.append(file)
    return missing_files


def splittimeframe(subreddit, start, end, split): # splits the time into a series of smaller files to pull
    split_list = []
    step = (end - start) / split
    for i in range(split):
            s = int(start + i * step)
            e = int((start + (i + 1) * step) - 1)
            if i == split - 1:
                    e += 86400
            split_list.append(f'{subreddit}-{s}.pickle')

    return split_list

def find_existing_pulls(type, subreddits): #remove existing pulls from subreddits list
    done = os.listdir(f'../../Files/{type}/score/')
    done.append([e for e in os.listdir(f'../../Files/{type}/')])
    for i in done:
        done[done.index(i)] = i[:-7]
    res = [i for i in subreddits if i not in done]
    return res

def merge_splits(files, subreddit, ptype): # merges the files down
    merge_candidates = []
    for file in files:
        try:
            if os.path.getsize(f'../../Files/{ptype}/temp/{file}') > 459:
                if file[:-18] == subreddit:
                    merge_candidates.append(f'../../Files/{ptype}/temp/{file}')
        except FileNotFoundError:
            pass

    merge_candidates.sort()
    if len(merge_candidates) == 24:
        df = pd.concat([pd.read_pickle(candidate) for candidate in merge_candidates])
        df.to_pickle(f'../../Files/{ptype}/{subreddit}.pickle')
        print(f'{subreddit} merged')
        for i in merge_candidates:
            os.remove(i)
    else:
        print(f'{subreddit} has only {len(merge_candidates)} files to merge. Merging not possible.')



In [7]:
with open(args.subreddits, newline='') as f:
    reader = csv.reader(f)
    subreddits = list(reader)

subreddits = list(chain.from_iterable(subreddits))

if args.job == 'Identify':
    subreddits_not_completed = find_existing_pulls(args.ptype, subreddits)

    start = int(datetime.datetime(2020, 3, 1).timestamp())
    end = int(datetime.datetime(2022, 3, 31).timestamp())
    split = 24
    missing_parts = []
    for subreddit in subreddits_not_completed:
        
        subreddit_parts = splittimeframe(subreddit, start, end, split)
        missing = identify_missing_files(f'../../Files/{args.ptype}/temp/', subreddit_parts)
        for m in missing:
            missing_parts.append(m)


    file = open(f'../../Files/{args.ptype}/temp/missing.csv', "w")
    writer = csv.writer(file, delimiter = "\n")
    for list_ in missing_parts:
        writer.writerow([list_])
    file.close()

if args.job == 'Merge':
    files = []
    files = os.listdir(f'../../Files/{args.ptype}/temp/')
    files.pop(files.index('missing.csv'))

    t = [file[:-18] for file in files]

    subreddits = []
    for i in t:
        if i not in subreddits:
            subreddits.append(i)

    for subreddit in subreddits:
        merge_splits(files, subreddit, args.ptype)

NameError: name 'args' is not defined

In [6]:
with open('../Preprocessing/subreddits_sm.csv', newline='') as f:
    reader = csv.reader(f)
    subreddits = list(reader)

subreddits = list(chain.from_iterable(subreddits))

In [11]:
subreddits_not_completed = find_existing_pulls('Comments', subreddits)

start = int(datetime.datetime(2020, 3, 1).timestamp())
end = int(datetime.datetime(2022, 3, 31).timestamp())
split = 24
missing_parts = []
for subreddit in subreddits_not_completed:
    
    subreddit_parts = splittimeframe(subreddit, start, end, split)
    missing = identify_missing_files(f'../../Files/Comments/temp/', subreddit_parts)
    for m in missing:
        missing_parts.append(m)

In [14]:
subreddits_not_completed = find_existing_pulls('Comments', subreddits)

In [21]:
type = "Comments"
done = os.listdir(f'../../Files/{type}/score/')
done.extend(os.listdir(f'../../Files/{type}/'))
for i in done:
    done[done.index(i)] = i[:-7]
res = [i for i in subreddits if i not in done]



In [22]:
done

['Government_is_lame',
 'WikiLeaks',
 'DemocraticSocialism',
 'weddingshaming',
 'collapse',
 'InternationalLeft',
 'aspiememes',
 'TheFightThatMatters',
 'LockdownCriticalLeft',
 'Astuff',
 'DebateVaccines',
 'POTUSWatch',
 'The_Ultimate',
 'libertarianmeme',
 'myanmar',
 'LouderWithCrowder',
 'Cringetopia',
 'AncientTruehistory',
 'AOC',
 'blursedimages',
 'ConservativeMemes',
 'conspiracyNOPOL',
 'GreenAndPleasant',
 'QAnonCasualties',
 'VACCINES',
 'aww',
 'ConspiracyUltra',
 'LateStageImperialism',
 'natureismetal',
 'wisconsin',
 'Labour',
 'NoNoNewNormal',
 'Slovakia',
 'AutisticPride',
 'altnewz',
 'Sino',
 'NEWPOLITIC',
 'CPUSA',
 'WomenInNews',
 'worldnewsvideo',
 'AmericanFascism2020',
 'EcoNewsNetwork',
 'OliverMarkusMalloy',
 'Anarchism',
 'ReallyAmerican',
 'thanksihateit',
 'NotoQanon',
 'didnthappen',
 'Africa',
 'thedavidpakmanshow',
 'KidsAreFuckingStupid',
 'PositiveNewsCovid19',
 'uninsurable',
 'uspolitics',
 'wokekids',
 'Palestine',
 'southafrica',
 'CapitolConse

In [2]:
df = pd.read_pickle('../../Files/Comments/score/POTUSWatch.pickle')

In [8]:
files = os.listdir('../../Files/Comments/score/')

In [9]:
files.remove('done')

In [10]:
tbd = []
for file in files:
    df = pd.read_pickle(os.path.join('../../Files/Comments/score/', file))
    if 'cleanBody' not in df.columns:
        tbd.append(file)

In [11]:
tbd

['myanmar.pickle']